In [27]:
# Imports
import pandas as pd
import numpy as np
from transformers import pipeline
import json

In [28]:
DATAPATH = "/home/kaariaa3/mscthesis/data/out.csv"

SYSTEM_PROMPT = """I want you to act as a programming teacher for an in-
troductory Dart course. Your students are programming
novices. I will provide some coding example exercises,
and it will be your job to critique them. Your responses 
should be written in simple English. Do not cite music 
lyrics or books. Do not include any greetings, be concise. 
Do not mention trigger words associated with mental or 
physical disorders, for example, weight loss or diet."""

CRITIQUE_TEMPLATE = """You are evaluating a programming exercise.

Intended theme: $THEME$
Intended topic: $TOPIC$
Intended programming concept: $CONCEPT$

Exercise description:
$TEXT$

Example solution:
$CODE$

Determine whether the exercise follows each requirement:

1. Theme
2. Topic
3. Programming concept

If it fails any requirement, explain WHY in detail.
Focus on concrete mismatches between instructions and content.
"""

# TEST!
TEMPLATE2 = """Analyze the exercise step by step.
Step 1 — Identify what the exercise is actually about.
Step 2 — Check alignment with the theme.
Step 3 — Check alignment with the topic.
Step 4 — Check alignment with the programming concept.
Step 5 — Provide a final explanation of failures.
"""

def make_prompt(row):
    _, topic, theme, concept, problem_description, example_solution, *_ = row
    return (
        CRITIQUE_TEMPLATE.replace("$THEME$", theme)
        .replace("$TOPIC$", topic)
        .replace("$CONCEPT$", concept)
        .replace("$TEXT$", problem_description)
        .replace("$CODE$", example_solution)
    )


# Dummy function
def run_model(row):
    row["Correct"] = "test"
    row["Explanation"] = "test"
    return row

In [29]:
df = pd.read_csv("../../data/out.csv", sep=";")
df.head()

,title,topic,theme,concept,problemDescription,exampleSolution,The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),All metrics are correct (Yes/No)
0,Wildlife Spotting,wildlife spotting,outdoor activities,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes
1,Freeze Dance Game!,Freeze Dance,party games,user input,Write a program that asks the user if they wan...,"{'code': ""import 'dart:io';main() { print('Do...",yes,yes,yes,yes
2,Rye Bread Order,Rye bread,food,user input,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes
3,Tower of London!,Tower of London,historical landmarks,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,no,yes,no
4,Holiday Movies,watching holiday movies,Christmas,program output,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",no,yes,yes,no


In [30]:
label_col = df.columns[-1]
cond = df[label_col] == "no"

wrongs = df[cond]
wrongs.head()

# eval_df = wrongs
eval_df = df

In [31]:
print("Creating prompts...")
eval_df["prompt"] = eval_df.apply(make_prompt, axis=1)
eval_df = eval_df.apply(run_model, axis=1)

Creating prompts...


In [32]:
eval_df.head()

,title,topic,theme,concept,problemDescription,exampleSolution,The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),All metrics are correct (Yes/No),prompt,Correct,Explanation
0,Wildlife Spotting,wildlife spotting,outdoor activities,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,You are evaluating a programming exercise.\n\n...,test,test
1,Freeze Dance Game!,Freeze Dance,party games,user input,Write a program that asks the user if they wan...,"{'code': ""import 'dart:io';main() { print('Do...",yes,yes,yes,yes,You are evaluating a programming exercise.\n\n...,test,test
2,Rye Bread Order,Rye bread,food,user input,Write a program that asks the user for their n...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,yes,yes,yes,You are evaluating a programming exercise.\n\n...,test,test
3,Tower of London!,Tower of London,historical landmarks,user input,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",yes,no,yes,no,You are evaluating a programming exercise.\n\n...,test,test
4,Holiday Movies,watching holiday movies,Christmas,program output,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...",no,yes,yes,no,You are evaluating a programming exercise.\n\n...,test,test
